In [1]:
from scripts.tools.issuetracker import clientapi as capi
import importlib
capi = importlib.reload(capi)
import threading
import datetime
import pytz

def _attrwalk(ob, path):
    for attr in path.split("."):
        ob = getattr(ob, attr)
    return ob

class RedmineProxyError(Exception):
    pass

def _filter_issues(issues, filters):
    out = {}
    for iid, iss in issues.items():
        b = True
        for v1, op, v2 in filters:
            v1 = _attrwalk(iss, v1)
            if op == "==":
                b = b and v1 == v2
            elif op == "!=":
                b = b and v1 != v2
            elif op == ">=":
                b = b and v1 >= v2
            elif op == "<=":
                b = b and v1 <= v2
            elif op == ">":
                b = b and v1 > v2
            elif op == "<":
                b = b and v1 < v2
            else:
                raise ValueError(op)
        if b:
            out[iid] = iss
    return out

class RedmineCache():
    def __init__(self):
        self._cache = {}
        self._updated = datetime.datetime(1970, 1, 1)
        
    def clear(self):
        self._cache.clear()
        
    def lookup(self, id):
        return self._cache[id]
    
    def get_all(self):
        return self._cache.copy()
    
    def update(self, issues):
        for iss in issues:
            self._cache[iss.id] = iss
        
    @property
    def last_update(self):
        return self._updated
    
    def filter(self, pred):
        out = {}
        for iid, iss in self._cache.items():
            try:
                res = pred(iss)
            except Exception:
                raise ValueError("Exception occurred while performing filter with custom predicate")
            out[iid] = iss
        return out
    
    def filter2(self, filters):
        return _filter_issues(self._cache, filters)
    
    def set_update_time(self, time=None):
        if time is None:
            self._updated = datetime.datetime.now(pytz.utc)
        else:
            assert isinstance(time, datetime.datetime)
            self._updated = time
        
class _StopManagerThread(Exception):
    pass

class RedmineCacheManager(threading.Thread):
    def __init__(self, update_interval=5, _cache=None):
        self._api = None
        self.lock = threading.Lock()  # just in case
        self._stop = threading.Event()
        self._cache = _cache or RedmineCache()
        self._update_interval = update_interval # seconds
        
        # It turns out that threading.Thread uses "_initialized" internally,
        # and I thought I was going crazy trying to find the bug in my code. 
        self.__initialized = False
        super().__init__(daemon=True)
        
    @property
    def api(self):
        # lazy / cached initialization
        if self._api is None:
            self._api = capi.IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
        return self._api

    def run(self):
        self._stop.clear()
        try:
            self._run()
        except _StopManagerThread:
            pass
        
    @property
    def is_initialized(self):
        return self.__initialized
        
    def _run(self):
        # on first iteration, populate cache with no delay
        wait = 0
        while True:
            # this strftime string gives an output that matches the one 
            # returned by the Redmine REST api for each issue. Testing showed
            # that it appeared to work correctly.
            updated_on = ">=" + self._cache.last_update.strftime("%Y-%m-%dT%H:%M:%SZ")
            iiter = self._do_download(updated_on)
            self._update_from_iter(iiter, 0)
            if self._stop.wait(self._update_interval):
                raise _StopManagerThread()
            
            wait = 1
            self.__initialized = True
            
    def _update_from_iter(self, iiter, wait):
        updated = False
        for issues in iiter:
            self._cache.update(issues)
            if self._stop.wait(wait):
                raise _StopManagerThread()
            updated = True
        if updated:
            self._cache.set_update_time()
    
    def _update_issues(self, issues):
        with self.lock:
            self._cache.update(issues)

    def _do_download(self, updated_on):
        try:
            return self.api.download_issues2(updated_on=updated_on, status_id="*")
        except Exception as e:
            raise RedmineProxyError("Error occurred collecting issues: \"%s\""%str(e))
        
    def stop(self):
        self._stop.set()
        
    def get_all(self):
        with self.lock:
            return self._cache.get_all()
    
    def lookup(self, id):
        with self.lock:
            return self._cache.lookup(id)
    
    def filter(self, filters):
        with self.lock:
            return self._cache.filter2(filters)

In [2]:
import socketserver
import http.server
import urllib
import base64
import pickle
import http
import socket
import select

def encode(item):
    b = pickle.dumps(item)
    return base64.b64encode(b)

def decode(string):
    b = base64.b64decode(string)
    return pickle.loads(b)

def parse_params(params):
    out = {}
    for k, v in params.items():
        if len(v) > 1:
            raise ValueError("Too many arguments for %r: %r" % (k, ", ".join(v)))
        out[k] = v[0]
    return out

class RedmineRequestHandler(http.server.SimpleHTTPRequestHandler):
    default_request_version = "HTTP/1.1"
    
    def handle(self):
        self.close_connection = False
        while not self.close_connection:
            # serve requests with a 10 minute timeout
            r, w, l = select.select([self.request], [], [], 60*10)
            if r:
                self.handle_one_request()
            else:
                break
    
    def do_GET(self):
        res = urllib.parse.urlparse(self.path) 
        qs = res.query
        params = urllib.parse.parse_qs(qs)
        params = parse_params(params)
        if res.path == "/cache":
            if not self.server.manager.is_initialized:
                return self.send_error(http.HTTPStatus.BAD_REQUEST, "Service is still initializing")
            self._do_cache_request(params)
        else:
            self.send_error(http.HTTPStatus.BAD_REQUEST, "Unknown path: %r" % res.path)
    
    def _do_cache_request(self, params):
        issues = None
        
        ireq = params.get('issues', 'all')
        if ireq == 'all':
            try:
                issues = self._do_request_all_issues()
            except Exception as e:
                return self.send_error(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Big Oops: %r"%str(e))
        else:
            l = ireq.split(",")
            try:
                issues = list(map(int, l))
            except ValueError:
                return self.send_error("invalid value for 'issues': %r"%issues)
            try:
                ret = [self.server.manager.lookup(i) for i in issues]
            except KeyError as e:
                return self.send_error(http.HTTPStatus.NOT_FOUND, "failed to find key: '%'" % str(e))
        
        filters = params.get('filters', None)
        if filters:
            filters = decode(filters)
            try:
                issues = _filter_issues(issues, filters)
            except ValueError:
                return self.send_error(http.HTTPStatus.BAD_REQUEST, "bad filter string: %r"%filters)
        
        if issues is None:
            return self.send_error(http.HTTPStatus.BAD_REQUEST, "Unknown args: %r" % params)
            
        rsp = encode(issues)
        self.send_response(200)
        self.send_header("Content-type", "text/plain")
        self.send_header("Content-Length", str(len(rsp)))
        if not self.close_connection:
            self.send_header("Connection", "Keep-Alive")
        else:
            self.send_header("Connection", "Close")
        self.end_headers()
        self.wfile.write(rsp)
        self.wfile.flush()
            
    def _do_request_all_issues(self):
        return self.server.manager.get_all()

class RedmineServer(socketserver.ThreadingTCPServer):
    
    def __init__(self, addr, manager=None):
        self.manager = manager or RedmineCacheManager()
        self.manager.start()
        super().__init__(addr, RedmineRequestHandler)
    
    # from http.server.HTTPServer
    def server_bind(self):
        """Override server_bind to store the server name."""
        socketserver.TCPServer.server_bind(self)
        host, port = self.socket.getsockname()[:2]
        self.server_name = socket.getfqdn(host)
        self.server_port = port
        
    def serve_forever(self, interval=0.05):
        # I attempted to speed up response time by overriding the default 
        # serve_forever with a shorter interval, but it doesn't seem to have worked.
        # oh well. 
        threading.Thread(target=lambda: super(self.__class__, self).serve_forever(interval), daemon=True).start()
    

In [3]:
try:
    manager.stop()
    manager = RedmineCacheManager(_cache=manager._cache)
except NameError:
    manager = RedmineCacheManager()
rs = RedmineServer(("localhost", 11649), manager)
rs.serve_forever(0.00)

Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                


127.0.0.1 - - [22/Mar/2019 14:26:34] "GET /cache?issues=all&filters=gANdcQBYEgAAAHByb2plY3QuaWRlbnRpZmllcnEBWAIAAAA9PXECWAsAAABwYnNzb2Z0d2FyZXEDh3EEYS4= HTTP/1.1" 200 -


Query returned 0 issues                


127.0.0.1 - - [22/Mar/2019 14:26:44] "GET /cache?issues=all&filters=gANdcQBYEgAAAHByb2plY3QuaWRlbnRpZmllcnEBWAIAAAA9PXECWAsAAABwYnNzb2Z0d2FyZXEDh3EEYS4= HTTP/1.1" 200 -


Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                


...

Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                


Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                


Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                


Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                


Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                
Query returned 0 issues                


In [ ]:
pass

In [ ]:
# import requests

# def decode(string):
#     b = base64.b64decode(string)
#     return pickle.loads(b)

# class RedmineClient():
#     _url = "http://localhost:11649/"
#     def __init__(self):
#         self.session = requests.Session()
#     def get_all(self):
#         r = self.session.get(self._url + "cache" + "?issues=all")
#         r.raise_for_status()
#         return decode(r.content.decode())

In [ ]:
# rs.shutdown()
# rs.manager.stop()
# rs.socket.close()

In [ ]:
# api = capi.IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')

# # capi = importlib.reload(capi)

# url = capi.uj(api._base_url, api._issues_url + ".json")
# now = datetime.datetime.now(pytz.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
# params = {"updated_on": ">=" + now}
# print(now)
# url += "?" + urllib.parse.urlencode(params)
# r = api._sess.get(url, auth=api._auth)

# url
# import json
# print()
# print(json.dumps(json.loads(r.content.decode()), indent=4))